In [1]:
pip install retry


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\user102\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import requests
from bs4 import BeautifulSoup
from retry import retry
import urllib
import time

In [3]:
import numpy as np

In [4]:
# 複数ページの情報をまとめて取得
data_samples = []

# スクレイピングするページ数
max_page = 10
# SUUMOを東京都23区のみ指定して検索して出力した画面のurl(ページ数フォーマットが必要)
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=25&pc=50&page={}'

# リクエストがうまく行かないパターンを回避するためのやり直し
@retry(tries=3, delay=10, backoff=2)
def load_page(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

# 処理時間を測りたい
start = time.time()
times = []

# ページごとの処理
for page in range(1,max_page+1):
    before = time.time()
    # ページ情報
    soup = load_page(url.format(page))
    # 物件情報リストを指定
    mother = soup.find_all(class_='cassetteitem')
    
    # 物件ごとの処理
    for child in mother:

        # 建物情報
        data_home = []
        # カテゴリ
        data_home.append(child.find(class_='ui-pct ui-pct--util1').text)
        # 建物名
        data_home.append(child.find(class_='cassetteitem_content-title').text)
        # 住所
        data_home.append(child.find(class_='cassetteitem_detail-col1').text)
        # 最寄り駅のアクセス
        children = child.find(class_='cassetteitem_detail-col2')
        for id,grandchild in enumerate(children.find_all(class_='cassetteitem_detail-text')):
            data_home.append(grandchild.text)
        # 築年数と階数
        children = child.find(class_='cassetteitem_detail-col3')
        for grandchild in children.find_all('div'):
            data_home.append(grandchild.text)

        # 部屋情報
        rooms = child.find(class_='cassetteitem_other')
        for room in rooms.find_all(class_='js-cassette_link'):
            data_room = []
            
            # 部屋情報が入っている表を探索
            for id_, grandchild in enumerate(room.find_all('td')):
                # 階
                if id_ == 2:
                    data_room.append(grandchild.text.strip())
                # 家賃と管理費
                elif id_ == 3:
                    data_room.append(grandchild.find(class_='cassetteitem_other-emphasis ui-text--bold').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--administration').text)
                # 敷金と礼金
                elif id_ == 4:
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--deposit').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--gratuity').text)
                # 間取りと面積
                elif id_ == 5:
                    data_room.append(grandchild.find(class_='cassetteitem_madori').text)
                    data_room.append(grandchild.find(class_='cassetteitem_menseki').text)
                # url
                elif id_ == 8:
                    get_url = grandchild.find(class_='js-cassette_link_href cassetteitem_other-linktext').get('href')
                    abs_url = urllib.parse.urljoin(url,get_url)
                    data_room.append(abs_url)
            # 物件情報と部屋情報をくっつける
            data_sample = data_home + data_room
            data_samples.append(data_sample)
    
    # 1アクセスごとに1秒休む
    time.sleep(1)
    
    # 進捗確認
    # このページの作業時間を表示
    after = time.time()
    running_time = after - before
    times.append(running_time)
    print(f'{page}ページ目：{running_time}秒')
    # 取得した件数
    print(f'総取得件数：{len(data_samples)}')
    # 作業進捗
    complete_ratio = round(page/max_page*100,3)
    print(f'完了：{complete_ratio}%')
    # 作業の残り時間目安を表示
    running_mean = np.mean(times)
    running_required_time = running_mean * (max_page - page)
    hour = int(running_required_time/3600)
    minute = int((running_required_time%3600)/60)
    second = int(running_required_time%60)
    print(f'残り時間：{hour}時間{minute}分{second}秒\n')

# 音を出す
def sound():
    !rundll32 user32.dll,MessageBeep
    
# 処理時間を測りたい
finish = time.time()
running_all = finish - start
print('総経過時間：',running_all)
sound()


1ページ目：6.7076637744903564秒
総取得件数：333
完了：10.0%
残り時間：0時間1分0秒

2ページ目：5.2096405029296875秒
総取得件数：448
完了：20.0%
残り時間：0時間0分47秒

3ページ目：6.344027280807495秒
総取得件数：770
完了：30.0%
残り時間：0時間0分42秒

4ページ目：5.957893133163452秒
総取得件数：1021
完了：40.0%
残り時間：0時間0分36秒

5ページ目：7.164606094360352秒
総取得件数：1372
完了：50.0%
残り時間：0時間0分31秒

6ページ目：6.284463167190552秒
総取得件数：1635
完了：60.0%
残り時間：0時間0分25秒

7ページ目：5.8441002368927秒
総取得件数：1820
完了：70.0%
残り時間：0時間0分18秒

8ページ目：5.626110792160034秒
総取得件数：2010
完了：80.0%
残り時間：0時間0分12秒

9ページ目：5.727685928344727秒
総取得件数：2177
完了：90.0%
残り時間：0時間0分6秒

10ページ目：6.3984644412994385秒
総取得件数：2446
完了：100.0%
残り時間：0時間0分0秒

総経過時間： 61.26619601249695


In [6]:
import csv

# CSVファイル名を指定
csv_file_name = 'suumo_data.csv'

# CSVファイルにデータを書き込む（エンコーディングを 'utf-8-sig' に変更）
with open(csv_file_name, mode='w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)

    # CSVのヘッダーを書き込む（必要に応じてカスタマイズ）
    writer.writerow(['カテゴリ', '建物名', '住所', '最寄り駅のアクセス1', '最寄り駅のアクセス2', '築年数', '階数', '階', '家賃', '管理費', '敷金', '礼金', '間取り', '面積', 'URL'])

    # 各行のデータを書き込む
    writer.writerows(data_samples)
